# Code for Quark Mass (Recusive Jackknife)

## General settings

In [ ]:
%pip install numpy scipy iminuit matplotlib scienceplots

In [1]:
import os
import numpy as np
from iminuit import Minuit
from iminuit.cost import LeastSquares

xyz_size = 32
t_size = 64
a = 0.090713
a_invrs = 2.1753

t_half = int(t_size / 2)
xyz_cube = int(xyz_size**3)

bin_count = 18

droot = "/Volumes/X6/data/ccbar"

gfix = ["gfix_C", "gfix_L"]
gauge = ["Coulomb", "Landau"]
chan = ["ps", "v"]
binID = ["BIN" + f"{i:02}" for i in range(1, 1 + bin_count)]

rj_path = "4pt/RJ"

for igfix in gfix:
    if not os.path.exists(f"{droot}/{igfix}/{rj_path}"):
        os.makedirs(f"{droot}/{igfix}/{rj_path}")


## Preparation

This step is to make [bin#] packs of samples that contains [bin# - 1] samples in each pack.

In [2]:
for igfix in gfix:
    a1plus_root = f"{droot}/{igfix}/4pt/a1plus"
    rj_root = f"{droot}/{igfix}/{rj_path}"

    for ibin in range(bin_count):
        pack_path = f"{rj_root}/sp{binID[ibin]}"
        if not os.path.exists(f"{pack_path}"):
            os.makedirs(f"{pack_path}")

        for ich in chan:
            for it in range(t_half):
                t = f"{it:+03}"

                ifnames = " ".join(
                    [
                        f"{a1plus_root}/{ich}.{t}.{igfix}.BIN{i:02}"
                        for i in range(1, 1 + bin_count)
                        if i != ibin + 1
                    ]
                )

                jksampleRJ_path = f"{pack_path}/jksampRJ"
                if not os.path.exists(f"{jksampleRJ_path}"):
                    os.makedirs(f"{jksampleRJ_path}")

                os.system(
                    f"bin/jre -l {xyz_cube} -d {jksampleRJ_path} {ifnames}"
                )

## Prepotential

In [3]:
for igfix in gfix:
    rj_root = f"{droot}/{igfix}/{rj_path}"
    for ibin in binID:
        pack_path = f"{rj_root}/sp{ibin}"

        ifpath = f"{pack_path}/jksampRJ"
        ofpath = f"{pack_path}/prevRJ"
        os.system(f"rm -rf {ofpath}")  # clear data
        os.makedirs(ofpath, exist_ok=True)  # make directory for output files

        os.system(f"bin/prev -n {xyz_size} -d {ofpath} {ifpath}/*")

## KS function (TI)

In [4]:
for igfix in gfix:
    rj_root = f"{droot}/{igfix}/{rj_path}"
    for ibin in range(bin_count):
        pack_path = f"{rj_root}/sp{binID[ibin]}"

        prev_path = f"{pack_path}/prevRJ"
        ofpath = f"{pack_path}/fks-tiRJ"
        os.system(f"rm -rf {ofpath}")  # clear data
        os.makedirs(ofpath, exist_ok=True)  # make directory for output files

        RJbin_list = [f"BIN{i:02}" for i in range(1, 1 + bin_count) if i != ibin + 1]

        for it in range(t_half):
            t = f"{it:+03}"

            for ibinRJ in RJbin_list:
                psfname = f"{prev_path}/ps.{t}.{igfix}.{ibinRJ}"
                vfname = f"{prev_path}/v.{t}.{igfix}.{ibinRJ}"
                ofname = f"{ofpath}/{t}.{igfix}.{ibinRJ}"

                os.system(
                    f"bin/fks-ti -n {xyz_size} -m 0.0483 -o {ofname} {psfname} {vfname}"
                )

In [5]:
# jackknife average
for ig in range(2):
    result_path = f"result/{gauge[ig]}/fks-tiRJ"
    os.system(f"rm -rf {result_path}")  # clear data
    os.makedirs(result_path, exist_ok=True)  # make directory for output files
    for ibin in binID:
        for it in range(t_half):
            t = f"{it:+03}"
            iflist = f"{droot}/{gfix[ig]}/4pt/RJ/sp{ibin}/fks-tiRJ/{t}.{gfix[ig]}.*"
            ofname = f"{result_path}/{t}.{ibin}"

            os.system(f"bin/mean -l {xyz_cube} -o {ofname} -jc {iflist}")

        # Convert all data to spherical coordinate
    os.system(
        f"bin/cart2sphr -n {xyz_size} -d {result_path} -p sphr -s txt {result_path}/*"
    )

### Fit

In [6]:
def mcc_fit(rmin, rmax, rawdf):

    def gaussian(x, A, B, C):
        return A * np.exp(-(x**2) / B) + C

    def gaussian2(x, A1, B1, A2, B2, C):
        return gaussian(x, A1, B1, 0) + gaussian(x, A2, B2, 0) + C

    para = {
        "A1": 1,
        "B1": 10,
        "A2": 10,
        "B2": 1,
        "C": -1,
    }

    # Fit
    rawdata = np.loadtxt(rawdf, dtype=np.float64)

    mask = (rawdata[:, 0] > rmin / a) & (rawdata[:, 0] < rmax / a)
    subdata = rawdata[mask]

    fitdata = subdata[np.argsort(subdata[:, 0])]

    fitsites = fitdata[:, 0]
    fitfks = fitdata[:, 1]
    fiterr = fitdata[:, 2]

    # Fit
    least_squares = LeastSquares(fitsites, fitfks, fiterr, gaussian2)  # type: ignore
    m = Minuit(least_squares, **para)  # type: ignore
    m.migrad()

    return -m.values["C"] * a_invrs


for igauge in gauge:
    M_array = []
    for ibin in binID:
        M_array.append(mcc_fit(0.01, 0.82, f"result/{igauge}/fks-tiRJ/sphr.+29.{ibin}.txt"))
    
    M_array = np.array(M_array)
    M_mean = np.mean(M_array)
    M_var = np.sqrt(np.var(M_array) * (bin_count - 1) / bin_count)

    print(f"M  =  {M_mean} ± {M_var}   ({igauge})")

M  =  1.9323275427628086 ± 0.00771881756259639   (Coulomb)
M  =  1.5217202014678106 ± 0.004632566488403369   (Landau)
